##### System Libraries

In [81]:
import os
import subprocess
import logging

##### Required Libraries

In [82]:
try:
    import scapy
except ModuleNotFoundError:
    os.system('pip install scapy')
    import scapy
try:
    import tqdm
except ModuleNotFoundError:
    os.system('pip install tqdm')
    import tqdm
from scapy.all import rdpcap
from tqdm import trange,tqdm

### Analyzer

#### Minor Function Breakdowns
---
##### Generate_Logger
Generates a logger with default parameters for logging critical components and debugging
##### List Initializer
Helper function to shorten and simplify initializing and re-initializing lists with values
##### PCAP_Input
Takes a pcap file as input and returns and stores it in memory

In [83]:
class Analyzer:
    if True:
         #Initialization of the logger
        logger=logging.getLogger
        stored_logger_level=[logging.DEBUG]

        stored_pcap_data=[]
    def __init__(self):
        if True:
            self.logger=self.Generate_Logger(filename='AnalysisCore.log')
    def Generate_Logger(self,filename='default_logger_name.log'): 
        logging.basicConfig(filename=filename,format='%(asctime)s - %(levelname)s: %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',filemode='w',level=self.stored_logger_level[0])
        logger=logging.getLogger()
        logger.debug('Logger initialized.')
        return logger
    def List_Initializer(self,input_list,input_param):
        input_list.clear()
        input_list.append(input_param)
    def PCAP_Input(self,path):
        self.logger.info('Reading file from path, '+path)
        pcap_data=rdpcap(path)
        self.logger.debug('Stored '+path+' data in stored_pcap_data.')
        self.List_Initializer(self.stored_pcap_data,pcap_data)
        return pcap_data

In [88]:
A=Analyzer()
A.PCAP_Input('PCAP Data/30S-Down-Spo-Stm-Dis--Up-Dis.pcap')


<30S-Down-Spo-Stm-Dis--Up-Dis.pcap: TCP:221679 UDP:1458 ICMP:0 Other:1>

In [89]:
for packet in A.stored_pcap_data[0][:10]:
    print(packet.payload.dst)

192.168.0.128
155.133.224.2
192.168.0.128
192.168.0.128
155.133.224.2
192.168.0.128
192.168.0.128
155.133.224.2
192.168.0.128
192.168.0.128


In [90]:
unique_ips=[]
self_ips=['192.168.0.128']
blocked_ips=[]
blocked_ips=self_ips
B=A.stored_pcap_data[0]

for packet in B:
    flag=0
    try:
        for ip in blocked_ips:
            if packet.payload.dst==ip:
                flag=1
        if flag==0:
            blocked_ips.append(packet.payload.dst)
            unique_ips.append(packet.payload.dst)
    except AttributeError:
        for ip in blocked_ips:
            if ip=='256.256.256.256':
                flag=1
        if flag==0:
            unique_ips.append('256.256.256.256')
unique_ips

['155.133.224.2',
 '23.212.164.219',
 '18.211.21.156',
 '162.159.138.234',
 '155.133.224.10',
 '162.159.136.234',
 '35.186.224.22',
 '35.213.130.189',
 '13.107.246.58',
 '155.133.225.21',
 '20.207.73.85',
 '57.144.141.32',
 '35.186.224.24',
 '140.82.112.26',
 '239.255.255.250',
 '72.25.64.32',
 '256.256.256.256',
 '35.186.224.40',
 '192.168.0.1',
 '51.21.215.65',
 '13.107.5.93',
 '104.199.241.202',
 '192.168.0.255',
 '35.186.224.45',
 '162.159.128.233',
 '140.82.113.21',
 '224.0.0.251',
 '52.168.117.171']

In [91]:
A.stored_pcap_data[0][2].payload.dst

'192.168.0.128'